In [2]:
import numpy as np
import fitsio
import os
import pandas as pd
import matplotlib.pyplot as plt
import sep
import xgboost as xgb
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split 
from sklearn.decomposition import PCA

# Optimization
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import GridSearchCV


# Metrics
from sklearn import metrics
from sklearn.metrics import mean_squared_error

In [3]:
datapath = '/Users/avery/seoproject/data2w/'
fpath = '/Users/avery/seoproject/filenames.csv'
fnames = pd.read_csv(fpath)
fn = fnames['filename']
expts = []

for f in fn:
    h = fitsio.read_header(datapath+f)
    et = h['EXPTIME']
    expts.append(et)

### creating training and test sets

In [33]:
for i in range(0,len(fn)):
    if meds[i]==np.nan:
        data = fitsio.read(datapath+fn[i])
        bkg = sep.Background(data)
        data_sub = data - bkg
        meds[i]= np.median(data)

In [4]:
meds = np.load('/Users/avery/seoproject/expbkgmeds5iter.npy')
# df = pd.read_csv('/Users/avery/seoproject/allparamdf.csv')
df = pd.read_csv('/Users/avery/seoproject/parapim.csv')

In [81]:
df = pd.read_csv('/Users/avery/seoproject/parapim.csv')

In [9]:
df

,Unnamed: 0,temp,rh,wind spd,wind dir,moon angle
0,0,39.1,95,0.0,0,21.617428
1,1,45.2,90,2.9,178,65.219098
2,2,53.1,63,2.9,190,44.898670
3,3,49.6,74,4.8,166,44.903420
4,4,47.2,94,0.0,64,79.117844
...,...,...,...,...,...,...
314,314,51.6,43,8.2,303,54.550208
315,315,36.8,92,0.3,100,80.408719
316,316,32.1,89,0.1,315,4.753472
317,317,33.8,93,0.0,295,31.961029


***only using StandardScaler***

In [63]:
df2 = df.drop('Unnamed: 0',axis=1)
df2 = df2.drop('wind dir',axis=1)
df2 = StandardScaler().fit_transform(df2)
xtrain, xtest, ytrain, ytest = train_test_split(df2, meds, test_size=0.15)

***using StandardScaler and PCA***

In [40]:
# df2 = df.drop('Unnamed: 0',axis=1)
# df2 = StandardScaler().fit_transform(df2)
pca = PCA(n_components=3)
df3 = pca.fit_transform(df2)
xtrain, xtest, ytrain, ytest = train_test_split(df3, meds, test_size=0.15)

### training the algorithm

#### standard scaler

In [16]:
from xgboost import XGBRegressor

In [79]:
df2 = df.drop('Unnamed: 0',axis=1)
df2 = df2.drop('wind dir',axis=1)
df2 = StandardScaler().fit_transform(df2)
xtrain, xtest, ytrain, ytest = train_test_split(df2, meds, test_size=0.15)

In [80]:
regressor = xgb.XGBRegressor()
regressor.fit(xtrain, ytrain)
ypred = regressor.predict(xtest)
stdmsr = mean_squared_error(ytest,ypred)

In [81]:
print(f'mean squared error: {mean_squared_error(ytest,ypred)}')
print(f'root mean squared error: {np.sqrt(mean_squared_error(ytest,ypred))}')

mean squared error: 0.5430536866188049
root mean squared error: 0.7369217872619629


In [82]:
regressor.save_model('/Users/avery/seoproject/standmse0543.json')

In [36]:
tr = xgb.Booster()
tr.load_model('/Users/avery/seoproject/stand275.json')

In [83]:
ytest

array([1.0126867e-04, 8.1781857e-04, 5.4355316e+00, 7.6017933e+00,
       5.1739645e+00, 4.7370017e-04, 6.6340639e-05, 4.5116301e-04,
       1.5006133e-04, 6.4880133e-04, 2.6140974e+00, 5.1887473e-04,
       6.0996647e+00, 6.1458122e-04, 2.7216382e+00, 4.0463858e+00,
       5.0187130e+00, 3.8289475e+00, 7.5781784e+00, 1.9133107e-03,
       1.7810942e-04, 6.2191521e-04, 1.6828245e-04, 4.8952864e-04,
       2.5725582e+00, 1.9791880e+00, 1.5030125e-04, 3.2258197e-04,
       5.0740544e-04, 8.1758806e-04, 7.6469512e+00, 6.3452078e-04,
       4.3106613e+00, 4.5814919e+00, 1.4560294e-04, 7.6127977e+00,
       1.2144428e-05, 6.3996710e-04, 1.6309459e-05, 1.7352711e-04,
       1.0539192e-04, 1.1518145e-03, 4.0317783e+00, 4.7802758e+00,
       5.1911349e+00, 1.0001987e+01, 8.8520616e-04, 9.9767504e+00],
      dtype=float32)

In [38]:
trp = tr.predict(xgb.DMatrix(xtest))
print(f'mean squared error: {mean_squared_error(ytest,trp)}')
print(f'root mean squared error: {np.sqrt(mean_squared_error(ytest,trp))}')

mean squared error: 7.569271087646484
root mean squared error: 2.7512307167053223


default parameters:
- ```eta``` or ```learning_rate```: step size shrinkage used in update to prevent overfitting; after each boosting step, we can directly get the weights of new features, and eta shrinks the feature weights to make the boosting process more conservative; range [0,1]; default 0.3
- ```gamma``` or ```min_split_loss```: minimum loss reduction required to make a further partition on a leaf node of the tree. the larger ```gamma``` is, the more conservative the algorithm will be; range [0, inf]; default 0
- ```max depth```: maximum depth of a tree. increasing this value will make the model more ocmplex and more likely to overfit. 0 indicates no limit on depth. beware that xgboost aggressively consumes memory when training a deep tree. ```exact``` tree method requires nonzero value; range [0,inf]; default 6
- ```min_child_weight```: minimum sum of instance weight (hessian) needed in a child. if the tree partition step results in a leaf node with the sum of instance weight less than ```min_child_weight```, then the building process will give up further partitioning. in linear regression task, this simply corresponds to minimum number of instances needed to be in each node. the larger ```min_child_weight``` is, the more conservative the algorithm will be. range [0,inf] default 1
- ```lambda``` or ```reg_lambda```: L2 regularization term on weights; increasing this value will make model more conservative; default 1

#### optimization

In [19]:
%%time

param_grid = {'n_estimators': [50,100,150,200], 'reg_lambda': [0.5,1,1.5],
             'gamma':[0,0.5,1], 'max_depth': [3,6,10,15]}
halving_cv = HalvingGridSearchCV(XGBRegressor(), param_grid, scoring="neg_mean_squared_error", n_jobs=-1, min_resources="exhaust", factor=3)

halving_cv.fit(xtrain,ytrain)

test = xgb.XGBRegressor(
    n_estimators=halving_cv.best_params_['n_estimators'],
    reg_lambda=halving_cv.best_params_['reg_lambda'],
    gamma=halving_cv.best_params_['gamma'],
    max_depth=halving_cv.best_params_['max_depth']
)

test.fit(xtrain,ytrain)
ypred = test.predict(xtest)
print(f'mean squared error: {mean_squared_error(ytest,ypred)}')

sthvmsr = mean_squared_error(ytest,ypred)

mean squared error: 8.179444313049316
CPU times: user 3.56 s, sys: 209 ms, total: 3.77 s
Wall time: 7.66 s


In [8]:
%%time
gridsearch = GridSearchCV(XGBRegressor(), param_grid, scoring="neg_mean_squared_error")
gridsearch.fit(xtrain, ytrain)
regressor = xgb.XGBRegressor(
    n_estimators=gridsearch.best_params_['n_estimators'],
    reg_lambda=gridsearch.best_params_['reg_lambda'],
    gamma=gridsearch.best_params_['gamma'],
    max_depth=gridsearch.best_params_['max_depth']
)

regressor.fit(xtrain,ytrain)
ypred = regressor.predict(xtest)
print(f'mean squared error: {mean_squared_error(ytest,ypred)}')

stgdmsr = mean_squared_error(ytest,ypred)

mean squared error: 22.50870704650879
CPU times: user 4min 28s, sys: 1.83 s, total: 4min 29s
Wall time: 34.5 s


In [9]:
stdmsr, sthvmsr, stgdmsr

(23.311676, 22.397202, 22.508707)

#### PCA

In [36]:
# df2 = df.drop('Unnamed: 0',axis=1)
# df2 = StandardScaler().fit_transform(df2)
pca = PCA(n_components=4)
df3 = pca.fit_transform(df2)
xtrain, xtest, ytrain, ytest = train_test_split(df3, meds, test_size=0.15)

In [37]:
regressor = xgb.XGBRegressor()
regressor.fit(xtrain, ytrain)
ypred = regressor.predict(xtest)

n4pcamsr = mean_squared_error(ytest,ypred)

In [38]:
print(f'mean squared error: {mean_squared_error(ytest,ypred)}')

mean squared error: 0.652540922164917


In [39]:
regressor.save_model('/Users/avery/seoproject/pcamse0653.json')

#### optimization

In [184]:
param_grid = {'n_estimators': [50,100,150,200], 'reg_lambda': [0.5,1,1.5],
             'gamma':[0,0.5,1], 'max_depth': [3,6,10,15]}
halving_cv = HalvingGridSearchCV(xgb.XGBRegressor(), param_grid, scoring="neg_mean_squared_error", n_jobs=-1, min_resources="exhaust", factor=3)
halving_cv.fit(xtrain,ytrain)
regressor = xgb.XGBRegressor(
    n_estimators=halving_cv.best_params_['n_estimators'],
    reg_lambda=halving_cv.best_params_['reg_lambda'],
    gamma=halving_cv.best_params_['gamma'],
    max_depth=halving_cv.best_params_['max_depth']
)

regressor.fit(xtrain,ytrain)
ypred = regressor.predict(xtest)
print(f'mean squared error: {mean_squared_error(ytest,ypred)}')

n4pcahvmsr = mean_squared_error(ytest,ypred)

KeyboardInterrupt: 

In [57]:
print(f'{n3pcahvmsr} at {halving_cv.best_params_}') #n_components = 3

1.0242241621017456 at {'gamma': 1, 'max_depth': 10, 'n_estimators': 100, 'reg_lambda': 1.5}


#### gridsearchcv

In [186]:
%%time

gridsearch = GridSearchCV(XGBRegressor(), param_grid, scoring="neg_mean_squared_error")

gridsearch.fit(xtrain, ytrain)

test = xgb.XGBRegressor(
    n_estimators=gridsearch.best_params_['n_estimators'],
    reg_lambda=gridsearch.best_params_['reg_lambda'],
    gamma=gridsearch.best_params_['gamma'],
    max_depth=gridsearch.best_params_['max_depth']
)

test.fit(xtrain,ytrain)
ypred = test.predict(xtest)
print(f'mean squared error: {mean_squared_error(ytest,ypred)}')

n4pcagdmsr = mean_squared_error(ytest,ypred)

mean squared error: 1.3833354711532593
CPU times: user 4min 32s, sys: 2.44 s, total: 4min 35s
Wall time: 35.6 s


In [78]:
tregressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=0.5,
    gamma=0.5,
    max_depth=6
)

tregressor.fit(xtrain,ytrain)
ypred = tregressor.predict(xtest)
print(f'mean squared error: {mean_squared_error(ytest,ypred)}')

mean squared error: 6.066430568695068


In [123]:
results = {'regressor type':['default', 'halvinggridsearchcv', 'gridsearchcv'],'StandardScaler': [stdmsr, sthvmsr, stgdmsr], '2-component PCA': [n2pcamsr, n2pcahvmsr, n2pcagdmsr],
           '3-component PCA': [n3pcamsr, n3pcahvmsr, n3pcagdmsr], '4-component PCA':[n4pcamsr, n4pcahvmsr, n4pcagdmsr]}



In [124]:
rdf = pd.DataFrame.from_dict(results)

In [127]:
rdf # mean squared error

,regressor type,StandardScaler,2-component PCA,3-component PCA,4-component PCA
0,default,23.311676,9.186532,1.103137,3.662132
1,halvinggridsearchcv,22.397202,9.335927,1.024224,3.405843
2,gridsearchcv,22.508707,9.700326,2.958178,3.888152


In [142]:
rdf.to_csv('/Users/avery/seoproject/rmsresults.csv')

In [141]:
rdf # root mean squared error

,regressor type,StandardScaler,2-component PCA,3-component PCA,4-component PCA
0,default,4.828217,3.030929,1.050303,1.913670
1,halvinggridsearchcv,4.732568,3.055475,1.012040,1.845493
2,gridsearchcv,4.744334,3.114535,1.719936,1.971840


## Dash

In [187]:
! pip install dash   
! pip install dash-html-components                                         
! pip install dash-core-components                                     
! pip install plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 23.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 29.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 KB 7.2 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.1.2
    Uninstalling Werkzeug-2.1.2:
      Successfully uninstalled Werkzeug-2.1.2


In [2]:
from dash import Dash, html, dcc
import plotly.express as px

In [3]:
app = Dash(__name__)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for your data.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
app = Dash(__name__)

app.layout = html.Div([
    html.H6("Receive median background value prediction."),
    html.Div([
        "Temperature (°F): ",
        dcc.Input(id='my-input', value='initial value', type='text')
    ]),
    html.Div([
        "Temperature (°F): ",
        dcc.Input(id='my-input', value='initial value', type='text')
    ]),
    html.Br(),
    html.Div(id='my-output'),

])


@callback(
    Output(component_id='my-output', component_property='children'),
    Input(component_id='my-input', component_property='value')
)
def update_output_div(input_value):
    return f'Output: {input_value}'


if __name__ == '__main__':
    app.run(debug=True)
